In [1]:
import pandas as pd
from PyKomoran import *
from PyKomoran import Komoran, DEFAULT_MODEL
import re
from konlpy.tag import Okt
from sklearn.cluster import DBSCAN
import numpy as np
from sentence_transformers import SentenceTransformer,util

In [2]:
review = pd.read_csv("merged_data_with_university.csv")

In [3]:
# 감성어 사전 불러오기 및 처리
sentiment_data = pd.read_csv('KnuSentiLex/SentiWord_Dict.txt', sep='\t', header=None, names=['단어', '극성'])
sentiment_dictionary = sentiment_data.set_index('단어')['극성'].to_dict()  # 사전 형태로 변환

# 형태소 분석

In [4]:
# 리뷰 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣\s]', '', text)  # 한글과 공백만 남기기
    return text.strip()

# 전처리 적용
review['리뷰 텍스트'] = review['리뷰 텍스트'].apply(lambda x: preprocess_text(str(x)))

In [15]:
okt = Okt()
# 명사 추출 함수
def extract_nouns(text):
    try:
        pos_list = okt.pos(text, norm=True, stem=True)
        nouns = [word for word, tag in pos_list if tag == 'Noun']
        return ' '.join(nouns)
    except Exception as e:
        print(f"Error processing text: {text} - {e}")
        return ''

# 형용사 추출 함수
def extract_adjectives(text):
    try:
        pos_list = okt.pos(text, norm=True, stem=True)
        adjectives = [word for word, tag in pos_list if tag == 'Adjective']
        return ' '.join(adjectives)
    except Exception as e:
        print(f"Error processing text: {text} - {e}")
        return ''

# 명사 컬럼 추가
review['명사'] = review['리뷰 텍스트'].apply(lambda x: extract_nouns(str(x)))

# 형용사 컬럼 추가
review['형용사'] = review['리뷰 텍스트'].apply(lambda x: extract_adjectives(str(x)))

# 결과 확인
print(review[['리뷰 텍스트', '명사', '형용사']].head())

                                              리뷰 텍스트  \
0  서울권에 있는 카페 거의 다가봤는데 개인적으로 등 티어입니다 친구들 회기오면 무조건...   
1  분위기도 좋은데 커피가 너무 맛있어요 디카페인커피는 경험상 일반 아메리카노보다 맛이...   
2  오랫만에 왔는데 여전히 커피도 맛있네요 시나몬카푸치노였는데 은은한 시나몬향과 살짝 ...   
3  한번 와보고 싶었던 곳인데 드디어 와봅니다\n판나코타 궁금했는데 이건 인 개 주문해...   
4  오늘 일카페\n동생이 경희대 근처 커피집 다 이쁘고 맛나다고 ㅋ\n지나가다 봐둔 카...   

                                                  명사  \
0  서울 카페 거의 다가 개인 등 티어 친구 회 기오 무조건 데리 심지어 명 멀리사 데...   
1  분위기 커피 디카 페인 커피 경험 일반 아메리카노 맛 안 밤 수 조용조 단골 듯 곳...   
2  만 커피 시나몬 카푸치노 시나몬 향 살짝 단맛 커피 오늘 비 더 자주 근처 병원 때...   
3  한번 보고 곳 판나코타 것 인 개 주문 요 순삭 다른 디저트 휘 낭시 초코 티그레 ...   
4  오늘 카페 동생 경희대 근처 커피집 맛 카페 또 저 고가 보기 동네 혼자 못 여기 ...   

                                                 형용사  
0  있다 이다 좋아하다 비싸다 같다 저렴하다 따뜻하다 중요하다 맛있다 싫어하다 좋아하다...  
1                좋다 맛있다 덜하다 맛있다 용하다 많다 맛있다 좋다 같다 맛있다  
2                           여전하다 맛있다 은은하다 그렇다 맛있다 있다  
3  궁금하다 부드럽다 적당하다 궁금하다 없다 아쉽다 강하다 좋아하다 아니다 그렇다 없다...  
4        이쁘다 재미있다 차분하다 좋다 굳다 굳다 좋아하다 신기하다 달콤하다 굳다 굳

In [16]:
# 저장
review.to_csv('review_with_nouns_adjectives.csv', index=False, encoding='utf-8-sig')

0    서울권에 있는 카페 거의 다가봤는데 개인적으로 등 티어입니다 친구들 회기오면 무조건...
1    분위기도 좋은데 커피가 너무 맛있어요 디카페인커피는 경험상 일반 아메리카노보다 맛이...
2    오랫만에 왔는데 여전히 커피도 맛있네요 시나몬카푸치노였는데 은은한 시나몬향과 살짝 ...
3    한번 와보고 싶었던 곳인데 드디어 와봅니다\n판나코타 궁금했는데 이건 인 개 주문해...
4    오늘 일카페\n동생이 경희대 근처 커피집 다 이쁘고 맛나다고 ㅋ\n지나가다 봐둔 카...
Name: 리뷰 텍스트, dtype: object